In [ ]:
# WORKING
import pandas as pd
import json
import requests
from dhis2 import Api

headers = {'Authorization': 'Basic your-id'}
api = Api("https://prep.mets.or.ug", "your-username", "your-password")

#unique_id_attribute_uid = "wk44sbe36Wm"  # Replace with the UID of your "Unique ID" attribute
unique_id_attribute_uid = "buT2Gf4F9Ae"  # Replace with the UID of your "Address" attribute

oumapper = pd.read_csv('facilitydatamove-kiziba.csv')

for i in oumapper.index:
    currentOrgId = oumapper['kpid'][i]
    rightfulOrgId = oumapper['hmisid'][i]

    try:
        teiList = api.get('trackedEntityInstances', params={
            'ou': currentOrgId, 
            'program': 'F5ZdOBuqlJh', 
            'fields': 'trackedEntityInstance,attributes[attribute,value]', 
            'skipPaging': 'true'
        })
        data = teiList.json().get("trackedEntityInstances")

        for item in data:
            teiId = item.get("trackedEntityInstance")
            attributes = item.get("attributes")

            # Find the "Unique ID" attribute and check the value
            for attribute in attributes:
                if attribute.get('attribute') == unique_id_attribute_uid:
                    unique_id = attribute.get('value')
                    #if unique_id and unique_id.startswith("KLG"):
                    if unique_id and unique_id.startswith("Rakai"):
                        # Perform the transfer logic (your original code here)

                        try:
                            result = api.put('tracker/ownership/transfer', params={'trackedEntityInstance': teiId, 'program': 'F5ZdOBuqlJh', 'ou': rightfulOrgId})
                            teiResult = api.get('trackedEntityInstances', params={'trackedEntityInstance': teiId, 'fields': '*'})
                            entity = teiResult.json()['trackedEntityInstances'][0]
                            tei_json = json.dumps(entity)
                            tei_json = tei_json.replace(currentOrgId, rightfulOrgId)
                            dict_entity = json.loads(tei_json)

                            try:
                                transfer = api.put('trackedEntityInstances/' + teiId, data=dict_entity)
                                print(teiId + " " + result.json()["message"])
                            except Exception as transfer_error:
                                print(f"Error transferring TEI {teiId}: {transfer_error}")
                        except Exception as result_error:
                            print(f"Error with TEI ownership transfer for {teiId}: {result_error}")

                    else:
                        print(f"TEI {teiId} skipped: Unique ID does not start with KLG")
                    break

        print("===========================================")
        print("All TEIs successfully transferred to " + rightfulOrgId)

    except Exception as list_error:
        print(f"Error fetching TEI list for orgId {currentOrgId}: {list_error}")
